In [1]:
import json

##### Cleaning up and annotation

tijolaco e mercadopopular strings can simply be joined.
institutoliberal e solidariedade need '\n' separation.

In [2]:
solidariedade = 'solidariedadesocialista.wordpress.com/corpus.json'
tijolaco = 'tijolaco.com.br/corpus.json'
mercadopopular = 'mercadopopular.org/corpus.json'
institutoliberal = 'institutoliberal.org.br/corpus.json'
ocafezinho = 'ocafezinho.com/corpus.json'
midiasmascara = 'midiasemmascara.com.br/corpus.json'

In [3]:
def joinBodyLines(filename, sep=''):
    outlist = []
    with open(filename, 'r') as corpus:
         for line in corpus:
             post = json.loads(line)
             try:
                text = post['body']
                text = sep.join(text)
                post['body'] = text
             except KeyError: pass
             outlist.append(post)

    return outlist

In [4]:
ocafezinhodata = joinBodyLines(ocafezinho, sep='\n')
msmdata = joinBodyLines(midiasmascara, sep='\n')

In [4]:
iliberaldata = joinBodyLines(institutoliberal, sep='\n')
mercadopdata = joinBodyLines(mercadopopular)
tijolacodata = joinBodyLines(tijolaco)
psoldata     = joinBodyLines(solidariedade, sep='\n')
wholedata = iliberaldata + mercadopdata + tijolacodata + psoldata

print(len(wholedata))
print(len(psoldata))


15933
53


In [0]:
test_post = wholedata[1300]
print(wholedata[0])
test_post['body']

We want to convert words to lower case, remove punctuation and remove the list of Portuguese stop words in nltk.corpus.stopwords.

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
%load_ext Cython

In [7]:
%%cython
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

cpdef str _lower_and_clean(str text):
     cdef list words
     cdef list nonstopwords
     words = wordpunct_tokenize(text.lower())
     nonstopwords = []
     for w in words:
         if not w in stopwords.words("portuguese"):
            nonstopwords.append(w)
     return(" ".join(nonstopwords))

cpdef dict lower_and_clean(dict dic):
     try:
         dic['body'] = _lower_and_clean(dic['body'])
     except KeyError:
         dic['body'] = ""
     return(dic)

In [8]:
%%time
ocafezinhodata = list(map(lower_and_clean, ocafezinhodata))
msmdata = list(map(lower_and_clean, msmdata))

CPU times: user 26min 43s, sys: 1min 38s, total: 28min 21s
Wall time: 28min 23s


In [189]:
%%time
iliberaldata = list(map(lower_and_clean, iliberaldata))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [190]:
%%time
mercadopdata = list(map(lower_and_clean, mercadopdata))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs


In [191]:
%%time
tijolacodf = list(map(lower_and_clean, tijolacodata))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


In [192]:
%%time
psoldf = list(map(lower_and_clean, psoldata))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


The next step is to create dataframes, annotating each text with their blogs, authors, dates and political orientation.

In [10]:
import pandas as pd

In [11]:
ocafezinhodf = pd.DataFrame(ocafezinhodata)
ocafezinhodf['blog'] = 'ocafezinho'
ocafezinhodf['pol'] = 'left'

msmdf = pd.DataFrame(msmdata)
msmdf['blog'] = 'msm'
msmdf['pol'] = 'right'

ocafezinhodf.to_csv('datasets/ocafezinho.tsv', sep='\t')
msmdf.to_csv('datasets/midiasemmascara.tsv', sep='\t')

In [198]:
%%time
iliberaldf = pd.DataFrame(iliberaldata)
mercadopdf = pd.DataFrame(mercadopdata)
tijolacodf = pd.DataFrame(tijolacodata)
psoldf = pd.DataFrame(psoldata)
iliberaldf['blog'] = 'institutoliberal'
iliberaldf['pol'] = 'right'
mercadopdf['blog'] = 'mercadopopular'
mercadopdf['pol'] = 'right'
tijolacodf['blog'] = 'tijolaco'
tijolacodf['pol']  = 'left'
psoldf['blog'] = 'solidariedadesocialista'
psoldf['pol'] = 'left'

CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 51.4 ms


In [203]:
%%time
iliberaldf.to_csv('datasets/institutoliberal.tsv', sep='\t')
mercadopdf.to_csv('datasets/mercadopopular.tsv', sep='\t')
tijolacodf.to_csv('datasets/tijolaco.tsv', sep='\t')
psoldf.to_csv('datasets/solidariedadesocialista.tsv', sep='\t')

CPU times: user 796 ms, sys: 28 ms, total: 824 ms
Wall time: 818 ms


In [11]:
import json
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
